In [7]:
import pandas as pd
import numpy as np
import copy

In [6]:
from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
def get_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)

In [10]:
def get_time(time_str):
#     v = int(data1["starttime"][i][11:13])
    int_time = int(time_str[11:13])
    if 0 <= int_time and int_time < 6:
        return "night"
    if 6 <= int_time and int_time < 12:
        return "morning"
    if 12 <= int_time and int_time < 18:
        return "afternoon"
    if 18 <= int_time and int_time <= 23:
        return "evening"

In [29]:
data = pd.read_csv('data/2018/201801-citibike-tripdata.csv')

In [30]:
print(len(data))

718994


In [31]:
data1 = data[0:1000]

In [32]:
data1["time"] = copy.deepcopy(data1["tripduration"])
data1["distance"] = copy.deepcopy(data1["start station latitude"])
data1.head(5)

c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,time,distance
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72,W 52 St & 11 Ave,40.767272,-73.993929,505,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1,970,40.767272
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72,W 52 St & 11 Ave,40.767272,-73.993929,3255,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1,723,40.767272
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72,W 52 St & 11 Ave,40.767272,-73.993929,525,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1,496,40.767272
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72,W 52 St & 11 Ave,40.767272,-73.993929,447,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1,306,40.767272
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,W 52 St & 11 Ave,40.767272,-73.993929,3356,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1,306,40.767272


In [33]:
for i in range(len(data1)):
    new_time = get_time(data1["starttime"][i])
    data1["time"][i] = new_time
    
    d = get_distance(data1["start station latitude"][i], data1["start station longitude"][i], data1["end station latitude"][i], data1["end station longitude"][i])
    data1["distance"][i] = d

c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
data1.head(10)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,time,distance
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72,W 52 St & 11 Ave,40.767272,-73.993929,505,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1,afternoon,2.08
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72,W 52 St & 11 Ave,40.767272,-73.993929,3255,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1,afternoon,1.86
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72,W 52 St & 11 Ave,40.767272,-73.993929,525,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1,afternoon,1.44
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72,W 52 St & 11 Ave,40.767272,-73.993929,447,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1,afternoon,0.84
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72,W 52 St & 11 Ave,40.767272,-73.993929,3356,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1,evening,1.13
5,1602,2018-01-01 21:31:54.1920,2018-01-01 21:58:36.3530,72,W 52 St & 11 Ave,40.767272,-73.993929,482,W 15 St & 7 Ave,40.739355,-73.999318,30106,Subscriber,1968,1,evening,3.14
6,722,2018-01-02 07:54:53.6460,2018-01-02 08:06:55.8720,72,W 52 St & 11 Ave,40.767272,-73.993929,228,E 48 St & 3 Ave,40.754601,-73.971879,32059,Subscriber,1978,1,morning,2.33
7,434,2018-01-02 08:06:14.3310,2018-01-02 08:13:28.7670,72,W 52 St & 11 Ave,40.767272,-73.993929,173,Broadway & W 49 St,40.760683,-73.984527,30525,Subscriber,1983,1,morning,1.08
8,366,2018-01-02 08:10:15.2550,2018-01-02 08:16:21.8960,72,W 52 St & 11 Ave,40.767272,-73.993929,479,9 Ave & W 45 St,40.760193,-73.991255,27439,Subscriber,1974,1,morning,0.82
9,324,2018-01-02 08:13:30.8960,2018-01-02 08:18:55.1200,72,W 52 St & 11 Ave,40.767272,-73.993929,450,W 49 St & 8 Ave,40.762272,-73.987882,30892,Subscriber,1947,1,morning,0.75


In [35]:
del data1["starttime"]
del data1["stoptime"]
del data1["start station id"]
del data1["start station name"]
del data1["end station id"]
del data1["end station name"]
del data1["bikeid"]
del data1["start station latitude"]
del data1["start station longitude"]
del data1["end station latitude"]
del data1["end station longitude"]

In [37]:
data1.head(15)

,tripduration,usertype,birth year,gender,time,distance
0,970,Subscriber,1992,1,afternoon,2.08
1,723,Subscriber,1969,1,afternoon,1.86
2,496,Subscriber,1956,1,afternoon,1.44
3,306,Subscriber,1974,1,afternoon,0.84
4,306,Subscriber,1992,1,evening,1.13
5,1602,Subscriber,1968,1,evening,3.14
6,722,Subscriber,1978,1,morning,2.33
7,434,Subscriber,1983,1,morning,1.08
8,366,Subscriber,1974,1,morning,0.82
9,324,Subscriber,1947,1,morning,0.75


In [40]:
from sklearn.utils import shuffle
data1 = shuffle(data1)

In [63]:
data1.head(15)

,tripduration,usertype,birth year,gender,time,distance
276,614,Subscriber,1974,1,evening,1.92
988,544,Subscriber,1981,1,afternoon,1.41
318,694,Subscriber,1989,1,evening,1.75
373,2096,Subscriber,1960,2,morning,0.00
570,527,Subscriber,1962,1,afternoon,1.44
774,639,Subscriber,1971,1,afternoon,2.09
574,743,Subscriber,1964,1,afternoon,1.75
300,1204,Subscriber,1971,1,afternoon,2.14
346,1725,Subscriber,1972,2,afternoon,4.12
233,877,Subscriber,1992,1,morning,2.09


In [57]:
k = int(0.8*len(data1))
x_train = data1["tripduration"][:k]
x_test = data1["tripduration"][k:]
y_train = data1["distance"][:k]
y_test = data1["distance"][k:]

In [58]:
print(len(x_train), len(x_test))

800 200


In [72]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [67]:
model = LinearRegression()

model.fit(list(map(lambda x:[x],x_train)), y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [70]:
predict = model.predict(list(map(lambda x:[x],x_test)))

In [71]:
print(model.coef_)

[0.00233556]


In [74]:
print(metrics.mean_squared_error(y_test, predict))

0.8184167788922787


In [76]:
print(metrics.r2_score(y_test, predict))

0.42605498431491684
